# Implementacija algoritma $AQC(p)$

Ovde ćemo prikazati kako se implementira prvi algoritam koji su predložili Dōng i Lín. Nasumično ćemo generisati retku matricu veličine $16 \times 16$, a onda i vektor $b$ dužine $16$, nakon čega ćemo prikazati kako se rešava sistem jednačina $A x = b$.

In [1]:
import scipy.sparse as sparse
import numpy as np

rows = 16
cols = 16
density = 0.1  # 10% non-zero elements

random_sparse_matrix = sparse.random(rows, cols, density=density, format='coo')
A = random_sparse_matrix.toarray()

In [2]:
from math import sqrt

b = np.random.rand(cols)
c = sqrt(sum(b*b))
b = 1/c * b

Matrica $A$ najverovatnije nije ermitska pozitivno definitna, pa koristimo recept koji su dali autori da se matrica $A$ najpre pretvori u ermitsku.

In [3]:
X = np.array([[0.0, 1.0], [1.0, 0.0]])
Y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
Z = np.array([[1.0, 0.0], [0.0, -1.0]])

Op, Om = 1/2.0 * (X + 1.0j * Y), 1/2.0 * (X - 1.0j * Y)

In [4]:
A1 = np.kron(Op, A) + np.kron(Om, A.conj().T)
b1 = np.kron(np.array([0, 1]), b)
N = 2*cols

print((A1 == A1.conj().T).all())

True


Sada generišemo Hamiltonijane $H_0$ i $H_1$. Koristićemo proceduru za slučaj kada $A$ nije pozitivno definitna.

In [5]:
plus = 1/sqrt(2.0) * np.array([1, 1])
Plus = plus.reshape(-1, 1) @ plus.reshape(1, 2)
minus = 1/sqrt(2.0) * np.array([1, -1])
B1 = b1.reshape(-1, 1) @ b1.reshape(1, N)

Qb = np.eye(2*N) - np.kron(Plus, B1)
H0 = np.kron(Op, np.kron(Z, np.eye(N)) @ Qb) + np.kron(Om, Qb @ np.kron(Z, np.eye(N)))
H1 = np.kron(Op, np.kron(X, A1) @ Qb) + np.kron(Om, Qb @ np.kron(X, A1))

Proverimo da je sve dobro tako što ćemo ubaciti sopstvene vrednosti uz $0$ od $H_0$.

In [6]:
print(H0 @ np.kron(np.kron(np.array([0, 1]), plus), b1).reshape(-1, 1))
print(H0 @ np.kron(np.kron(np.array([1, 0]), minus), b1).reshape(-1, 1))

[[ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 8.32667268e-17+0.j]
 [ 2.77555756e-17+0.j]
 [ 6.93889390e-17+0.j]
 [ 0.00000000e+00+0.j]
 [ 1.11022302e-16+0.j]
 [ 1.38777878e-17+0.j]
 [ 1.24900090e-16+0.j]
 [ 1.11022302e-16+0.j]
 [ 6.93889390e-18+0.j]
 [ 1.38777878e-16+0.j]
 [ 8.32667268e-17+0.j]
 [ 1.38777878e-16+0.j]
 [ 8.32667268e-17+0.j]
 [ 8.67361738e-19+0.j]
 [ 5.55111512e-17+0.j]
 [ 1.38777878e-17+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j]
 [ 0.000000

Sve je tu negde na oko $10^{-16}$. To je verovatno zbog preciznosti računanja sa **float** promenljivama na računaru. Deluje da je ovo sve u redu.

Sada je potrebno definisati funkciju za protok vremena. Ona zavisi od kondicionog broja matrice $A1$. Iskoristićemo VQE da izračunamo aproksimaciju tog broja.

In [7]:
from scipy.optimize import minimize
 
from qiskit.circuit.library import efficient_su2
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
 
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

service = QiskitRuntimeService(channel = "local")
backend = service.least_busy()
service

hamiltonian = SparsePauliOp.from_operator(A1)

ansatz = efficient_su2(hamiltonian.num_qubits)

num_params = ansatz.num_parameters

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
 
ansatz_isa = pm.run(ansatz)

hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)

def trenutni_minimum(params, ansatz, hamiltonian, estimator):

    # ovaj deo koda generiše argumente za estimator
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    # u dictionary ubacujemo vrednosti dosadašnjih energija
    
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(
        f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]"
    )
 
    return energy

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

x0 = 2 * np.pi * np.random.random(num_params)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000
 
    res = minimize(
        trenutni_minimum,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla",
        options = {'maxiter' : 30, 'disp' : False}
    )

eig_min = min(cost_history_dict["cost_history"])

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

x0 = 2 * np.pi * np.random.random(num_params)

def trenutni_maksimum(params, ansatz, hamiltonian, estimator):
    return -1 * trenutni_minimum(params, ansatz, hamiltonian, estimator)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000
 
    res = minimize(
        trenutni_maksimum,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla",
        options = {'maxiter' : 30, 'disp' : False}
    )

eig_max = max(cost_history_dict["cost_history"])

kappa = np.abs(eig_max / eig_min)

Iters. done: 1 [Current cost: 0.1779343639182385]
Iters. done: 2 [Current cost: 0.01452238641153093]
Iters. done: 3 [Current cost: 0.007655829428685914]
Iters. done: 4 [Current cost: -0.005990026315568703]
Iters. done: 5 [Current cost: -0.009175097538744677]
Iters. done: 6 [Current cost: 0.10556299175314872]
Iters. done: 7 [Current cost: -0.05251632937114566]
Iters. done: 8 [Current cost: 0.006200755666822716]
Iters. done: 9 [Current cost: -0.04595087958313168]
Iters. done: 10 [Current cost: -0.04548400848019746]
Iters. done: 11 [Current cost: -0.06361117012562306]
Iters. done: 12 [Current cost: -0.08510808479341118]
Iters. done: 13 [Current cost: -0.13695436630073746]
Iters. done: 14 [Current cost: -0.054335686444893984]
Iters. done: 15 [Current cost: -0.20606281986745875]
Iters. done: 16 [Current cost: -0.04017155873241814]
Iters. done: 17 [Current cost: -0.10214687423492362]
Iters. done: 18 [Current cost: -0.19460274471110167]
Iters. done: 19 [Current cost: -0.07240790563767255]
Ite

In [8]:
kappa

0.9212571577421015

Sada imamo kondicioni broj i možemo definisati funkciju vremena.

In [9]:
def time_function(kappa, s, p):
    return kappa/(kappa - 1) * (1 - (1 + s*(kappa**(p - 1) - 1))**(1/(1 - p)))

Za parametar $p$ ćemo uzeti $0.5$. Sada biramo $T = O(\kappa / \epsilon)$. Neka to bude $T = 2 \kappa / \epsilon$, a za $\epsilon = 1/40$.

In [10]:
epsilon = 1.0/40.0
T = 2 * kappa / epsilon

T

73.70057261936812

Ostaje samo da se izabere parametar $M$ koji određuje koliko ćemo imati operatora vremenske evolucije. On se bira tako da bude $M = O(\text{polylog}(N) T^2 / \epsilon)$. Postavićemo ga na $10$ u ovom slučaju, čisto da bismo testirali kako radi na kvantnom računaru.

In [38]:
M = 10

Broj kubita je $\log_2$ od veličine matrice $H_0$. U ovom slučaju je to $7$. Sada pravimo kvantno kolo koje će opisati vremensku evoluciju. Na to kvantno kolo najpre treba inicijalizovati $\ket{0} \ket{-} \ket{b_1}$.

In [39]:
from qiskit.circuit import QuantumCircuit
from math import log2
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit.library import StatePreparation

# predstavljamo H0 i H1 u SparsePauliOp

h0, h1 = SparsePauliOp.from_operator(H0), SparsePauliOp.from_operator(H1)
n = int(log2(H0.shape[0]))

time_evolution = QuantumCircuit(n)

initial_state = np.kron(np.array([1, 0]), np.kron(minus, b1))
stateprep = StatePreparation(initial_state)
time_evolution.append(stateprep, range(n))

h = 1.0/M

for i in reversed(range(1, M + 1)):
    si = i*h
    time_evolution.append(PauliEvolutionGate(h1, time = T * time_function(kappa, si, 0.5) * h), range(n))
    time_evolution.append(PauliEvolutionGate(h0, time = T * (1 - time_function(kappa, si, 0.5)) * h), range(n))

time_evolution.measure_all()

Ostaje sad samo da se odredi vektor $\ket{x''}$.

In [40]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(mode=backend)

qc = pm.run(time_evolution)

result = sampler.run([(qc, [])]).result()

print(result)

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=1024, num_bits=7>)), metadata={'shots': 1024, 'circuit_metadata': {}})], metadata={'version': 2})


In [41]:
print(result[0].data.meas.get_bitstrings()[:10])

['0101110', '0001001', '1000111', '0010001', '0001110', '0000000', '1100001', '1110110', '0001001', '0010010']


In [44]:
counts_int = result[0].data.meas.get_int_counts()
counts_bin = result[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

y = np.array([sqrt(final_distribution_100_int[x]) for x in range(2**n)])

print(y)
print(sqrt(sum(y * y)))

[0.07654655 0.10364452 0.09882118 0.0625     0.11267348 0.06987712
 0.08838835 0.10364452 0.09882118 0.10825318 0.08267973 0.09375
 0.05412659 0.08838835 0.08838835 0.05412659 0.08267973 0.07654655
 0.11692679 0.0625     0.07654655 0.08838835 0.08838835 0.07654655
 0.05412659 0.08838835 0.0625     0.09375    0.08267973 0.09375
 0.08838835 0.10364452 0.08267973 0.08267973 0.09375    0.08267973
 0.10825318 0.0625     0.09375    0.08838835 0.09375    0.10825318
 0.08838835 0.07654655 0.10364452 0.10364452 0.11692679 0.08267973
 0.08267973 0.08838835 0.07654655 0.08267973 0.08838835 0.08267973
 0.12103073 0.06987712 0.08838835 0.09882118 0.07654655 0.10825318
 0.09882118 0.09375    0.08267973 0.06987712 0.09375    0.09882118
 0.07654655 0.06987712 0.07654655 0.08838835 0.09375    0.07654655
 0.09882118 0.0625     0.09375    0.07654655 0.07654655 0.09375
 0.0625     0.04419417 0.08838835 0.07654655 0.09375    0.09882118
 0.08838835 0.07654655 0.08838835 0.09375    0.08267973 0.10364452
 0.0

Sada je potrebno izvući vektor $x''$ iz ovoga. Ovaj vektor koji smo dobili je u obliku $\ket{0}\ket{+}\ket{x''}$. Kada izračunamo $\ket{0}\ket{+}$, dobijamo $(1/\sqrt{2}, 1/\sqrt{2}, 0, 0)^T$. Onda je vektor koji smo dobili u obliku $((1/\sqrt{2}) x'', (1/\sqrt{2}) x'', 0, 0)^T$. Dakle, čitanjem prvih $N$ koordinata i množenjem sa $\sqrt{2}$ se može dobiti vektor $x''$.

In [45]:
x_guess = np.array([sqrt(2.0) * y[i] for i in range(N)])
x_guess

array([0.10825318, 0.14657549, 0.13975425, 0.08838835, 0.15934436,
       0.09882118, 0.125     , 0.14657549, 0.13975425, 0.15309311,
       0.11692679, 0.13258252, 0.07654655, 0.125     , 0.125     ,
       0.07654655, 0.11692679, 0.10825318, 0.16535946, 0.08838835,
       0.10825318, 0.125     , 0.125     , 0.10825318, 0.07654655,
       0.125     , 0.08838835, 0.13258252, 0.11692679, 0.13258252,
       0.125     , 0.14657549])

Sada množimo $A_1$ i $x''$ da bismo dobili konstantu $d$.

In [55]:
temp = (A1 @ x_guess.reshape(-1, 1)).reshape(1, N)

In [58]:
d = 0

for i in range(N):
    if temp[0, i] != 0:
        d += b1[i] / temp[0, i]

d /= N

print(d)

print(1/d * b1)
print(temp)

(8.90084396637771+0j)
[0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.03506384+0.j 0.01203539+0.j 0.03409924+0.j 0.0147322 +0.j
 0.03580178+0.j 0.00692153+0.j 0.03734206+0.j 0.04066278+0.j
 0.02262036+0.j 0.03427407+0.j 0.02280264+0.j 0.04190873+0.j
 0.0253139 +0.j 0.0006546 +0.j 0.02695029+0.j 0.01266312+0.j]
[[0.        +0.j 0.2167009 +0.j 0.01161676+0.j 0.00754249+0.j
  0.        +0.j 0.12437042+0.j 0.17005163+0.j 0.27109612+0.j
  0.16905263+0.j 0.16458649+0.j 0.13629289+0.j 0.        +0.j
  0.        +0.j 0.30741398+0.j 0.        +0.j 0.20213994+0.j
  0.00324119+0.j 0.09798587+0.j 0.23519568+0.j 0.15400509+0.j
  0.        +0.j 0.26746244+0.j 0.11538486+0.j 0.        +0.j
  0.00125921+0.j 0.02334482+0.j 0.12016307+0.j 0.11771039+0.j
  0.15563305+0.j 0.04687021+0.j 0.09755235+0.j 0.332942

Konačno, dobijamo: $x = c * d * x''$.

In [61]:
x = c * d * x_guess

print(x)

[2.32626102+0.j 3.14977232+0.j 3.00319007+0.j 1.89938417+0.j
 3.42416351+0.j 2.12357606+0.j 2.68613486+0.j 3.14977232+0.j
 3.00319007+0.j 3.28982989+0.j 2.51264908+0.j 2.84907626+0.j
 1.64491495+0.j 2.68613486+0.j 2.68613486+0.j 1.64491495+0.j
 2.51264908+0.j 2.32626102+0.j 3.55342241+0.j 1.89938417+0.j
 2.32626102+0.j 2.68613486+0.j 2.68613486+0.j 2.32626102+0.j
 1.64491495+0.j 2.68613486+0.j 1.89938417+0.j 2.84907626+0.j
 2.51264908+0.j 2.84907626+0.j 2.68613486+0.j 3.14977232+0.j]


Ova aproksimacija je katastrofalna, jer smo izabrali malo $M$.